# Study of frequency analysis (not presented in the paper)

In this study, a unique event class is randomly generated at different time. For a short perdio of time, the frequency of the event changes (from once every few days to multiple times a day). Without any further indication, memorability identifies the frequency change as memorable, given that event that occur frequently require more information to be identified than rare events (compare "the only event last week" with "the 13th event of 3 days ago").

In [4]:
import sys
sys.path.append("../")

In [5]:
import pandas as pd
from abduction_memorability.event import Event, Label
from abduction_memorability.abduction_module import SurpriseAbductionModule
from abduction_memorability.predicate import Predicate
from abduction_memorability.memory import Memory
from abduction_memorability.predicate_filter import OptimizedFilter
from abduction_memorability.helpers import Helpers

from typing import Tuple
import random
import datetime as dt

import plotly.io as pio
import plotly.express as px
import plotly.offline as py

In [6]:
root_label = Label()

all_events: list[Event] = []

CURRENT_DAY = 200
EPOCH_SHIFT = 1577836800
MORE_FREQUENT = 123     # when the events become more frequent


LOW_CHANCE = 1/(24*60*5)
HIGH_CHANCE = 100 * LOW_CHANCE

In [7]:
first_today = True
for day in range(0, CURRENT_DAY):
    first_today = True
    for hour in range(24):
        for minute in range(60):
            chance = random.random()
            if chance < LOW_CHANCE and day != MORE_FREQUENT and first_today:
                event = Event(
                    timestamp=day*86400 + hour * 3600 + minute * 60 + EPOCH_SHIFT,
                    duration=10,
                    characteristics={},
                    label=root_label
                )
                all_events.append(event)
                first_today = False
            if chance < HIGH_CHANCE and day == MORE_FREQUENT:
                event = Event(
                    timestamp=day*86400 + hour * 3600 + minute * 60 + EPOCH_SHIFT,
                    duration=10,
                    characteristics={},
                    label=root_label
                )
                all_events.append(event)
                first_today = False

In [8]:
len(all_events)


59

In [9]:
class Day(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)

    def __call__(self, event: Event):
        if self._prog is None:
            return None
        if self._prog > CURRENT_DAY:
            return None
        e_time = event.timestamp - EPOCH_SHIFT
        return (e_time // 86400) == CURRENT_DAY - self._prog

    def program_length(self):
        return Helpers.bit_length(CURRENT_DAY)
    
    def __str__(self):
        return f"Day({self._prog})"

class Hour(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)
    
    def __call__(self, event: Event):
        if self._prog is None:
            return None
        if self._mem.last_date() - self._mem.first_date() > dt.timedelta(days=1):
            return None
        if self._prog not in range(24):
            return None
        e_time = event.timestamp - EPOCH_SHIFT
        return ((e_time % 86400) // 3600) == self._prog

    def program_length(self):
        return Helpers.bit_length(24)

    def __str__(self):
        return f'Hour({self._prog})'

class Minute(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)
    
    def __call__(self, event: Event):
        if self._prog is None:
            return None
        if self._mem.last_date() - self._mem.first_date() > dt.timedelta(hours=1):
            return None
        if self._prog not in range(60):
            return None
        e_time = event.timestamp - EPOCH_SHIFT
        return (((e_time % 86400) % 3600) // 60) == self._prog

    def program_length(self):
        return Helpers.bit_length(60)

    def __str__(self):
        return f'Minute({self._prog})'


class Time(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)

    def __call__(self, event: Event):
        if self._prog is None:
            return None
        if self._prog in range(CURRENT_DAY):
            return Day(self.mem, self._prog)(event)
        if self._prog in range(CURRENT_DAY, CURRENT_DAY*24):
            day = self._prog / 24
            hour = self._prog % CURRENT_DAY
            return Day(self._mem, day)(event) and Hour(self._mem, hour)(event)
        
    

In [10]:
mem = Memory(all_events)

In [11]:
# How many events in the remarkable day ?
print(len(OptimizedFilter(Day(mem, 77))(mem)))

25


In [12]:
import datetime as dt
print(mem.last_date())
print(mem.first_date())
print(mem.last_date() - mem.first_date() > dt.timedelta(days=1))

2020-07-18 11:11:00
2020-01-04 02:30:00
True


In [13]:
module = SurpriseAbductionModule(mem,
    predicates=[
        Day,
        Hour,
        Minute
    ])

Loaded the memory with 59 items!
Computing complexities with 4 passes
Starting pass 0 with 1 memories to explore
Finished pass 0 in 0.01273655891418457s.
Improved complexity for 34 event(s)
Starting pass 1 with 1 memories to explore
Finished pass 1 in 0.01133108139038086s.
Improved complexity for 9 event(s)
Starting pass 2 with 7 memories to explore
Finished pass 2 in 0.029204130172729492s.
Improved complexity for 16 event(s)
Computing surprise scores for all events !


In [14]:
df = module.dataframe_output()
df

,id,time,label,complexity,recipe,memorability,date
0,56,1594409640,event,19.452169,[Day(9)],11.040887,2020-07-10 19:34:00
1,36,1588525740,event,34.585189,"[Day(77), Hour(17)]",3.643242,2020-05-03 17:09:00
2,53,1592335020,event,19.452169,[Day(33)],11.040887,2020-06-16 19:17:00
3,31,1588516260,event,51.653115,"[Day(77), Hour(14), Minute(31)]",9.186844,2020-05-03 14:31:00
4,3,1579410360,event,19.452169,[Day(182)],11.040887,2020-01-19 05:06:00
5,5,1580101560,event,19.452169,[Day(174)],11.040887,2020-01-27 05:06:00
6,45,1589448120,event,19.452169,[Day(66)],11.040887,2020-05-14 09:22:00
7,55,1592556600,event,19.452169,[Day(30)],11.040887,2020-06-19 08:50:00
8,49,1590656700,event,19.452169,[Day(52)],11.040887,2020-05-28 09:05:00
9,12,1583096640,event,19.452169,[Day(140)],11.040887,2020-03-01 21:04:00


In [16]:
fig = px.scatter(df, x="date", y="complexity", hover_data=['recipe'], height=800)
fig.update_traces(marker={'size':10})
fig.update_layout(
    showlegend=False,
    font=dict(
        size=25
    )
)